# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [29]:
import numpy as np
import math
import torch
from torch import nn
from torch.utils.cpp_extension import load
import torch.nn.functional as F
from collections import OrderedDict
from time import sleep
from tqdm import tqdm

torch.__version__

'1.0.0'

## Custom C++ & CUDA modules

In [2]:
import peephole_lstm_classes

c:\users\0107w\anaconda3\envs\pytorch\lib\site-packages\torch\utils\cpp_extension.py:92: UserWarning: Error checking compiler version: Command '['c++']' returned non-zero exit status 1.
  warnings.warn('Error checking compiler version: {}'.format(error))
c:\users\0107w\anaconda3\envs\pytorch\lib\site-packages\torch\utils\cpp_extension.py:118: UserWarning: 

                               !! WARNING !!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Your compiler (c++) may be ABI-incompatible with PyTorch!
Please use a compiler that is ABI-compatible with GCC 4.9 and above.
See https://gcc.gnu.org/onlinedocs/libstdc++/manual/abi.html.

See https://gist.github.com/goldsborough/d466f43e8ffc948ff92de7486c5216d6
for instructions on how to install GCC 4.9 or higher.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                              !! WARNING !!

  warnings.warn(ABI_INCOMPATIBILITY_WARNING.format(compiler))


In [3]:
dropout = 0.
class PeepholeNetworkCPP(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeLSTMCell(129, 300)
        self.lstm1 = peephole_lstm_classes.PeepholeLSTMCell(300, 300)
        self.lstm2 = peephole_lstm_classes.PeepholeLSTMCell(300, 300)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        output = x.new_empty(x.size(0), x.size(1), 129)
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        for i, seq_batch in enumerate(x.transpose(0, 1)):
            hc1 = self.lstm0(seq_batch, hc1)
            #states = self.lstm0(seq_batch, states)
            hc2 = self.lstm1(F.dropout(hc1[0], p=0, training=self.training), hc2)
            hc3 = self.lstm2(F.dropout(hc2[0], p=0, training=self.training), hc3)
            output[:, i, :] = self.fc(F.dropout(hc3[0], p=0, training=self.training))
            #output[:, i, :] = self.fc(F.dropout(states[0], p=0, training=self.training))
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return output, new_states
        #return output, states
class PeepholeNetworkTorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeLSTMCellTorch(129, 300)
        self.lstm1 = peephole_lstm_classes.PeepholeLSTMCellTorch(300, 300)
        self.lstm2 = peephole_lstm_classes.PeepholeLSTMCellTorch(300, 300)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        output = x.new_empty(x.size(0), x.size(1), 129)
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        for i, seq_batch in enumerate(x.transpose(0, 1)):
            hc1 = self.lstm0(seq_batch, hc1)
            #states = self.lstm0(seq_batch, states)
            hc2 = self.lstm1(F.dropout(hc1[0], p=0, training=self.training), hc2)
            hc3 = self.lstm2(F.dropout(hc2[0], p=0, training=self.training), hc3)
            output[:, i, :] = self.fc(F.dropout(hc3[0], p=0, training=self.training))
            #output[:, i, :] = self.fc(F.dropout(states[0], p=0, training=self.training))
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return output, new_states
        #return output, states
class PeepholeLoopTorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeLSTMTorch(129, 300, 0)
        self.lstm1 = peephole_lstm_classes.PeepholeLSTMTorch(300, 300, 0)
        self.lstm2 = peephole_lstm_classes.PeepholeLSTMTorch(300, 300, 0)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states
class PeepholeLoop(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeLSTM(129, 300, 0)
        self.lstm1 = peephole_lstm_classes.PeepholeLSTM(300, 300, 0)
        self.lstm2 = peephole_lstm_classes.PeepholeLSTM(300, 300, 0)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        #x, states = self.lstm0(x, states)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states
        #return x, states
class PeepholeNormTorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeNormLSTMTorch(129, 300, 0)
        self.lstm1 = peephole_lstm_classes.PeepholeNormLSTMTorch(300, 300, 0)
        self.lstm2 = peephole_lstm_classes.PeepholeNormLSTMTorch(300, 300, 0)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states


device = ['cpu', 'cuda:0'][1]

# model_cpp = peephole_lstm_cell_classes.PeepholeLSTMCell(129, 300)
# model_torch = peephole_lstm_cell_classes.PeepholeLSTMCellTorch(129, 300)
model_cpp = PeepholeNetworkCPP()
model_torch = PeepholeNetworkTorch()
model_torch_loop = PeepholeLoopTorch()
model_loop = PeepholeLoop()

model_norm_torch = PeepholeNormTorch()

for module_name in ('lstm0', 'lstm1', 'lstm2', 'fc'):
#for module_name in ('lstm0', 'fc'):
    copyfrom = getattr(model_cpp, module_name)
    if module_name != 'fc':
        mod = getattr(model_torch, module_name)
        mod.weight_ih.data = copyfrom.weight_ih.data
        mod.weight_hh.data = copyfrom.weight_hh.data
        mod.weight_ch.data = copyfrom.weight_ch.data
        mod.bias.data = copyfrom.bias.data
        mod = getattr(model_loop, module_name)
        mod.weight_ih.data = copyfrom.weight_ih.data
        mod.weight_hh.data = copyfrom.weight_hh.data
        mod.weight_ch.data = copyfrom.weight_ch.data
        mod.bias.data = copyfrom.bias.data
        mod = getattr(model_torch_loop, module_name)
        mod.weight_ih.data = copyfrom.weight_ih.data
        mod.weight_hh.data = copyfrom.weight_hh.data
        mod.weight_ch.data = copyfrom.weight_ch.data
        mod.bias.data = copyfrom.bias.data
    else:
        mod = getattr(model_torch, module_name)
        mod.weight.data = copyfrom.weight.data
        mod.bias.data = copyfrom.bias.data
        mod = getattr(model_loop, module_name)
        mod.weight.data = copyfrom.weight.data
        mod.bias.data = copyfrom.bias.data
        mod = getattr(model_torch_loop, module_name)
        mod.weight.data = copyfrom.weight.data
        mod.bias.data = copyfrom.bias.data

model_cpp.to(device)
model_torch.to(device)
model_torch_loop.to(device)
model_loop.to(device)
model_norm_torch.to(device)

batch_size = 32
seq_length = 1

play_input = torch.randn(batch_size, seq_length, 129).to(device)
play_states = (torch.zeros(3, batch_size, 300).to(device), torch.zeros(3, batch_size, 300).to(device))
#play_states = torch.zeros(4, 600).to(device).chunk(dim=1, chunks=2)

# print(model_cpp)
# print(model_torch)
# print(model_torch_loop)
# print(model_loop)
# print(model_norm_torch)

In [6]:
n0 = peephole_lstm_classes.PeepholeNormLSTMTorch(129, 300, dropout=0., momentum=0.1, eps=1e-8)
n1 = peephole_lstm_classes.NormalizedPeepholeLSTM(129, 300, dropout=0., momentum=0.1, eps=1e-8)

for params in zip(n0.parameters(), n1.parameters()):
    params[0].data = params[1].data

n0.cuda()
n1.cuda()

# print(n0.running_var_tanh_cell)
# print(n1.running_var_tanh_cell)
# print("#####################################################")
# print(n0(play_input, (play_states[0][0], play_states[1][0])))
# print('-----------------------------------------------------')
# print(n1(play_input, (play_states[0][0], play_states[1][0])))
# print("#####################################################")
# print(n0.running_var_tanh_cell)
# print(n1.running_var_tanh_cell)

NormalizedPeepholeLSTM(input_size=129, hidden_size=300, dropout=0.0, momentum=0.1, eps=1e-08)

In [7]:
n0.zero_grad()
n1.zero_grad()
p0, p1 = play_input.clone().requires_grad_(True), play_input.clone().requires_grad_(True)
noise = torch.randn(batch_size, 1, 300, device=device)

t = n0(p0, (play_states[0][0], play_states[1][0]))[0].mul(noise).sum()
print(t)
t.backward()
t = n1(p1, (play_states[0][0], play_states[1][0]))[0].mul(noise).sum()
print(t)
t.backward()
print()

print(n0.weight_ih.grad[::50])
print(n1.weight_ih.grad[::50])
print(p0.grad)
print(p1.grad)

tensor(-2.6410, device='cuda:0', grad_fn=<SumBackward0>)
tensor(-2.6410, device='cuda:0', grad_fn=<SumBackward0>)

tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [ -0.9028,  -0.7898,   1.1119,  ...,  -1.9372,   3.4961,   1.6380],
        [  6.0501,   7.9203,   2.0289,  ...,   7.1978,  -5.3366,   6.3599],
        [ 21.1133, -19.8142,   6.5822,  ...,  10.6429,   0.4151,  -5.9493]],
       device='cuda:0')
tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [ -0.9028,  -0.7898,   1.1119,  ...,  -1.9372,   3.4961,   1.6379],
        [  6.0501,   7.9203,   2.0289,  ...,   7.1978,  -5.3366,   6.3599]

In [12]:
%%timeit -n 1 -r 200
n0(play_input, (play_states[0][0], play_states[1][0]))

2.12 ms ± 267 µs per loop (mean ± std. dev. of 1000 runs, 1 loop each)


In [11]:
%%timeit -n 1 -r 200
n1(play_input, (play_states[0][0], play_states[1][0]))

8.76 ms ± 1.06 ms per loop (mean ± std. dev. of 1000 runs, 1 loop each)


In [14]:
%%timeit -n 1 -r 200 out = n0(play_input, (play_states[0][0], play_states[1][0]))[0]
n0.zero_grad()
out.pow(2).sum().backward()

4.34 ms ± 971 µs per loop (mean ± std. dev. of 200 runs, 1 loop each)


In [15]:
%%timeit -n 1 -r 200 out = n1(play_input, (play_states[0][0], play_states[1][0]))[0]
n1.zero_grad()
out.pow(2).sum().backward()

8.63 ms ± 879 µs per loop (mean ± std. dev. of 200 runs, 1 loop each)


Vanila PyTorch implementation

In [4]:
%%timeit -n 1 -r 30
out = model_torch(play_input, play_states)[0]

493 ms ± 50.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [5]:
%%timeit -n 1 -r 30 out = model_torch(play_input, play_states)[0]
model_torch.zero_grad()
out.pow(2).sum().backward()

896 ms ± 24.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [6]:
model_torch.eval()
model_torch.zero_grad()
out = model_torch(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_torch.lstm0.weight_ih.grad.data.sum().add(model_torch.lstm0.weight_hh.grad.data.sum()).add(model_torch.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[ 0.0562,  0.0598,  0.0121,  ..., -0.0333, -0.0352, -0.0040],
         [ 0.0576,  0.0620,  0.0103,  ..., -0.0361, -0.0360, -0.0049],
         [ 0.0576,  0.0631,  0.0094,  ..., -0.0379, -0.0359, -0.0053],
         ...,
         [ 0.0582,  0.0649,  0.0088,  ..., -0.0394, -0.0347, -0.0060],
         [ 0.0579,  0.0650,  0.0090,  ..., -0.0392, -0.0347, -0.0060],
         [ 0.0577,  0.0653,  0.0091,  ..., -0.0389, -0.0349, -0.0060]],

        [[ 0.0558,  0.0600,  0.0130,  ..., -0.0329, -0.0354, -0.0040],
         [ 0.0575,  0.0622,  0.0111,  ..., -0.0361, -0.0365, -0.0049],
         [ 0.0582,  0.0637,  0.0100,  ..., -0.0380, -0.0373, -0.0052],
         ...,
         [ 0.0575,  0.0636,  0.0074,  ..., -0.0397, -0.0349, -0.0064],
         [ 0.0578,  0.0634,  0.0074,  ..., -0.0398, -0.0345, -0.0064],
         [ 0.0578,  0.0635,  0.0078,  ..., -0.0394, -0.0345, -0.0066]],

        [[ 0.0560,  0.0601,  0.0124,  ..., -0.0335, -0.0353, -0.0036],
         [ 0.0578,  0.0626,  0.0106,  ..., -

In [7]:
sleep(3)
torch.cuda.empty_cache()

Loop within each module

In [8]:
%%timeit -n 1 -r 30
out = model_torch_loop(play_input, play_states)[0]

404 ms ± 19.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [9]:
%%timeit -n 1 -r 30 out = model_torch_loop(play_input, play_states)[0]
model_torch_loop.zero_grad()
out.pow(2).sum().backward()

896 ms ± 34.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [10]:
model_torch_loop.eval()
model_torch_loop.zero_grad()
out = model_torch_loop(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_torch_loop.lstm0.weight_ih.grad.data.sum().add(model_torch_loop.lstm0.weight_hh.grad.data.sum()).add(model_torch_loop.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[ 0.0562,  0.0598,  0.0121,  ..., -0.0333, -0.0352, -0.0040],
         [ 0.0576,  0.0620,  0.0103,  ..., -0.0361, -0.0360, -0.0049],
         [ 0.0576,  0.0631,  0.0094,  ..., -0.0379, -0.0359, -0.0053],
         ...,
         [ 0.0582,  0.0649,  0.0088,  ..., -0.0394, -0.0347, -0.0060],
         [ 0.0579,  0.0650,  0.0090,  ..., -0.0392, -0.0347, -0.0060],
         [ 0.0577,  0.0653,  0.0091,  ..., -0.0389, -0.0349, -0.0060]],

        [[ 0.0558,  0.0600,  0.0130,  ..., -0.0329, -0.0354, -0.0040],
         [ 0.0575,  0.0622,  0.0111,  ..., -0.0361, -0.0365, -0.0049],
         [ 0.0582,  0.0637,  0.0100,  ..., -0.0380, -0.0373, -0.0052],
         ...,
         [ 0.0575,  0.0636,  0.0074,  ..., -0.0397, -0.0349, -0.0064],
         [ 0.0578,  0.0634,  0.0074,  ..., -0.0398, -0.0345, -0.0064],
         [ 0.0578,  0.0635,  0.0078,  ..., -0.0394, -0.0345, -0.0066]],

        [[ 0.0560,  0.0601,  0.0124,  ..., -0.0335, -0.0353, -0.0036],
         [ 0.0578,  0.0626,  0.0106,  ..., -

In [11]:
sleep(3)
torch.cuda.empty_cache()

Single Peephole LSTM Cell C++ (Loop implemented with Python)

In [12]:
%%timeit -n 1 -r 30
out = model_cpp(play_input, play_states)[0]

418 ms ± 29.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [13]:
%%timeit -n 1 -r 30 out = model_cpp(play_input, play_states)[0]
model_cpp.zero_grad()
out.pow(2).sum().backward()

805 ms ± 29.9 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [14]:
model_cpp.eval()
model_cpp.zero_grad()
out = model_cpp(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_cpp.lstm0.weight_ih.grad.data.sum().add(model_cpp.lstm0.weight_hh.grad.data.sum()).add(model_cpp.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[ 0.0562,  0.0598,  0.0121,  ..., -0.0333, -0.0352, -0.0040],
         [ 0.0576,  0.0620,  0.0103,  ..., -0.0361, -0.0360, -0.0049],
         [ 0.0576,  0.0631,  0.0094,  ..., -0.0379, -0.0359, -0.0053],
         ...,
         [ 0.0582,  0.0649,  0.0088,  ..., -0.0394, -0.0347, -0.0060],
         [ 0.0579,  0.0650,  0.0090,  ..., -0.0392, -0.0347, -0.0060],
         [ 0.0577,  0.0653,  0.0091,  ..., -0.0389, -0.0349, -0.0060]],

        [[ 0.0558,  0.0600,  0.0130,  ..., -0.0329, -0.0354, -0.0040],
         [ 0.0575,  0.0622,  0.0111,  ..., -0.0361, -0.0365, -0.0049],
         [ 0.0582,  0.0637,  0.0100,  ..., -0.0380, -0.0373, -0.0052],
         ...,
         [ 0.0575,  0.0636,  0.0074,  ..., -0.0397, -0.0349, -0.0064],
         [ 0.0578,  0.0634,  0.0074,  ..., -0.0398, -0.0345, -0.0064],
         [ 0.0578,  0.0635,  0.0078,  ..., -0.0394, -0.0345, -0.0066]],

        [[ 0.0560,  0.0601,  0.0124,  ..., -0.0335, -0.0353, -0.0036],
         [ 0.0578,  0.0626,  0.0106,  ..., -

In [15]:
sleep(3)
torch.cuda.empty_cache()

For-Loop implemented inside C++

Custom Kernal

In [16]:
%%timeit -n 1 -r 30
out = model_loop(play_input, play_states)[0]

65.7 ms ± 7.98 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [17]:
%%timeit -n 1 -r 30 out = model_loop(play_input, play_states)[0]
model_loop.zero_grad()
out.pow(2).sum().backward()

51.3 ms ± 1.73 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [18]:
model_loop.eval()
model_loop.zero_grad()
out = model_loop(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_loop.lstm0.weight_ih.grad.data.sum().add(model_loop.lstm0.weight_hh.grad.data.sum()).add(model_loop.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[ 0.0562,  0.0598,  0.0121,  ..., -0.0333, -0.0352, -0.0040],
         [ 0.0576,  0.0620,  0.0103,  ..., -0.0361, -0.0360, -0.0049],
         [ 0.0576,  0.0631,  0.0094,  ..., -0.0379, -0.0359, -0.0053],
         ...,
         [ 0.0582,  0.0649,  0.0088,  ..., -0.0394, -0.0347, -0.0060],
         [ 0.0579,  0.0650,  0.0090,  ..., -0.0392, -0.0347, -0.0060],
         [ 0.0577,  0.0653,  0.0091,  ..., -0.0389, -0.0349, -0.0060]],

        [[ 0.0558,  0.0600,  0.0130,  ..., -0.0329, -0.0354, -0.0040],
         [ 0.0575,  0.0622,  0.0111,  ..., -0.0361, -0.0365, -0.0049],
         [ 0.0582,  0.0637,  0.0100,  ..., -0.0380, -0.0373, -0.0052],
         ...,
         [ 0.0575,  0.0636,  0.0074,  ..., -0.0397, -0.0349, -0.0064],
         [ 0.0578,  0.0634,  0.0074,  ..., -0.0398, -0.0345, -0.0064],
         [ 0.0578,  0.0635,  0.0078,  ..., -0.0394, -0.0345, -0.0066]],

        [[ 0.0560,  0.0601,  0.0124,  ..., -0.0335, -0.0353, -0.0036],
         [ 0.0578,  0.0626,  0.0106,  ..., -

In [19]:
sleep(3)
torch.cuda.empty_cache()

Normalized - Torch

In [4]:
%%timeit -n 1 -r 30
out = model_norm_torch(play_input, play_states)[0]

487 ms ± 60.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [5]:
%%timeit -n 1 -r 30 out = model_norm_torch(play_input, play_states)[0]
model_norm_torch.zero_grad()
out.pow(2).sum().backward()

918 ms ± 44.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [6]:
model_norm_torch.eval()
model_norm_torch.zero_grad()
out = model_norm_torch(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_norm_torch.lstm0.weight_ih.grad.data.sum().add(model_norm_torch.lstm0.weight_hh.grad.data.sum()).add(model_norm_torch.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[-0.0704, -0.0244,  0.0305,  ..., -0.0407,  0.0059, -0.0575],
         [-0.0381,  0.0100,  0.0460,  ..., -0.0236,  0.0547, -0.0603],
         [-0.0108,  0.0340,  0.0947,  ...,  0.0006,  0.0875, -0.0402],
         ...,
         [-0.2807, -0.1009,  0.0343,  ...,  0.0449, -0.1193, -0.0647],
         [-0.2771, -0.0293, -0.0023,  ...,  0.0099, -0.1284, -0.1171],
         [-0.3189, -0.0023, -0.0334,  ..., -0.0255, -0.0703, -0.1567]],

        [[-0.0681, -0.0217,  0.0038,  ..., -0.0490,  0.0505, -0.0682],
         [-0.1424, -0.0186, -0.0091,  ..., -0.0620,  0.0914, -0.0968],
         [-0.1891,  0.0523, -0.0042,  ..., -0.0220,  0.0909, -0.0717],
         ...,
         [ 0.2524,  0.0960,  0.3093,  ...,  0.0566,  0.0381, -0.1388],
         [ 0.2102,  0.0286,  0.2966,  ...,  0.0631,  0.0219, -0.1538],
         [ 0.1145, -0.0168,  0.2166,  ...,  0.0050, -0.0295, -0.1838]],

        [[-0.0538, -0.0228,  0.0727,  ..., -0.0062,  0.0874, -0.0723],
         [-0.0777, -0.0269,  0.0551,  ...,  

In [7]:
sleep(3)
torch.cuda.empty_cache()

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([35, 53, 26,  1, 45, 70, 54, 77, 14,  6,  6,  6, 25, 26,  1,  1, 48,
       77, 79, 26, 81, 55, 67, 55, 70,  0, 77, 26, 54, 70, 77, 26, 67, 67,
       77, 26, 67, 55, 11, 70, 15, 77, 70, 42, 70, 54, 48, 77, 19, 73, 53,
       26,  1,  1, 48, 77, 79, 26, 81, 55, 67, 48, 77, 55,  0, 77, 19, 73,
       53, 26,  1,  1, 48, 77, 55, 73, 77, 55, 45,  0, 77, 75, 69, 73,  6,
       69, 26, 48,  2,  6,  6, 21, 42, 70, 54, 48, 45, 53, 55, 73])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
#     n_batches = ((len(arr) - batch_size) // seq_length) // batch_size
    n_batches = ((len(arr) - 1) // seq_length) // batch_size
    
    ## TODO: Keep only enough characters to make full batches
#     arr = arr[:batch_size * (seq_length * n_batches + 1)]
    arr = np.concatenate([arr[seq * seq_length * n_batches:(seq + 1) * seq_length * n_batches + 1]
                          for seq in range(batch_size)])
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size, seq_length * n_batches + 1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
#     for n in range(0, arr.shape[1], seq_length):
    for n in range(0, arr.shape[1] - 1, seq_length):
        # The features
        x = arr[:, n:n +  seq_length]
        # The targets, shifted by one
        y = arr[:, n + 1:n + seq_length + 1]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[35 53 26  1 45 70 54 77 14  6]
 [ 0 75 73 77 45 53 26 45 77 26]
 [70 73 23 77 75 54 77 26 77 79]
 [ 0 77 45 53 70 77 63 53 55 70]
 [77  0 26 69 77 53 70 54 77 45]
 [63 19  0  0 55 75 73 77 26 73]
 [77 62 73 73 26 77 53 26 23 77]
 [40 34 67 75 73  0 11 48  2 77]]

y
 [[53 26  1 45 70 54 77 14  6  6]
 [75 73 77 45 53 26 45 77 26 45]
 [73 23 77 75 54 77 26 77 79 75]
 [77 45 53 70 77 63 53 55 70 79]
 [ 0 26 69 77 53 70 54 77 45 70]
 [19  0  0 55 75 73 77 26 73 23]
 [62 73 73 26 77 53 26 23 77  0]
 [34 67 75 73  0 11 48  2 77 36]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

# DO IT ALL (Runs all the previous cells except for the custom modules)

In [11]:
print("DONE")

DONE


---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [12]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')
# train_on_gpu = False

Training on GPU!


In [13]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(input_size=len(tokens), hidden_size=n_hidden, num_layers=n_layers,
                            batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(in_features=n_hidden, out_features=len(tokens))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size, cuda=True):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if cuda:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0., lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        in_size = len(tokens)
        for i in range(n_layers):
            self.add_module(f'lstm{i}', peephole_lstm_classes.PeepholeLSTM(in_size, n_hidden, drop_prob))
            in_size = n_hidden
        #self.lstm = peephole_lstm_classes.PeepholeLSTM(len(tokens), n_hidden, drop_prob)
        self.fc = nn.Linear(in_features=in_size, out_features=len(tokens))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        new_hidden = (torch.zeros_like(hidden[0]), torch.zeros_like(hidden[1]))
        for i in range(self.n_layers):
            x, hc = self.__getattr__(f'lstm{i}')(x, (hidden[0][i], hidden[1][i]))
            new_hidden[0][i], new_hidden[1][i] = hc[0], hc[1]
        x = self.fc(x)
        
        # return the final output and the hidden state
        return x, new_hidden
    
    
    def init_hidden(self, batch_size, cuda=True):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if cuda:
            hidden = (weight.new_zeros(self.n_layers, batch_size, self.n_hidden).cuda(),
                      weight.new_zeros(self.n_layers, batch_size, self.n_hidden).cuda())
        else:
            hidden = (weight.new_zeros(self.n_layers, batch_size, self.n_hidden),
                      weight.new_zeros(self.n_layers, batch_size, self.n_hidden))
        
        return hidden

In [15]:
class MyCharRNN(nn.Module):
    
    class PeepholeLSTMCell(nn.Module):
        
        def __init__(self, input_size, hidden_size, dropout=0):
            assert isinstance(input_size, int) and input_size >= 1
            assert isinstance(hidden_size, int) and hidden_size >= 1
            assert isinstance(dropout, (int, float)) and 0 <= dropout <= 1

            super().__init__()

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.dropout = dropout
            
            self.weight_ih = nn.Parameter(torch.empty(4*hidden_size, input_size))
            self.weight_hh = nn.Parameter(torch.empty(4*hidden_size, hidden_size))
            self.weight_ch = nn.Parameter(torch.empty(3*hidden_size, hidden_size))
            self.bias = nn.Parameter(torch.empty(4*hidden_size))
            
            self.reset_parameters()
            
        def reset_parameters(self):
            stdv = 1. / np.sqrt(self.input_size + 2 * self.hidden_size)
            self.weight_ih.data.uniform_(-stdv, stdv)
            self.weight_hh.data.uniform_(-stdv, stdv)
            self.weight_ch.data.uniform_(-stdv, stdv)
            self.bias.data.uniform_(-stdv, stdv)
            
        def forward(self, input, hc=None):
            if hc is None:
                hc = input.new_zeros(input.size(1), 2 * self.hidden_size, requires_grad=False).chunk(chunks=2, dim=1)

            for i, h in enumerate((hc[0], hc[1])):
                if input.size(0) != h.size(0):
                    raise RuntimeError(f"Input batch size {input.size(0)} doesn't match hidden{i} batch size {h.size(0)}")
                if h.size(1) != self.hidden_size:
                    raise RuntimeError(f"hidden{i} has inconsistent hidden_size: got {h.size(1)}, expected {self.hidden_size}")
            
            gates = torch.addmm(self.bias, input, self.weight_ih.t())
            gates += torch.mm(hc[0], self.weight_hc.t())
            gates[:, :3*self.hidden_size] += torch.mm(hc[1], self.weight_ch.t())
            
            sig_gates = gates[:, :3 * self.hidden_size].sigmoid().chunk(chunks=3, dim=1)
            tanh_gate = gates[:, 3 * self.hidden_size:].tanh()
            
            c = sig_gates[0] * hc[1] + sig_gates[1] * tanh_gate
            h = sig_gates[2] * torch.tanh(c)
            output = F.dropout(h, p=self.dropout, training=self.training, inplace=False)
            
            return output, (h, c)
            
        def __repr__(self):
            return f"PeepholeLSTMCell(input_size={self.input_size}, hidden_size={self.hidden_size}, dropout={self.dropout})"
        
        def extra_repr(self):
            return ''

    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the layers of the model
        self.peephole_lstm0 = self.PeepholeLSTMCell(input_size=len(tokens), hidden_size=n_hidden, dropout=drop_prob)
        for n in range(1, n_layers):
            self.add_module(f"peephole_lstm{n}", self.PeepholeLSTMCell(input_size=n_hidden, hidden_size=n_hidden, dropout=drop_prob))
        self.fc = nn.Linear(in_features=n_hidden, out_features=len(tokens))
    
    def forward(self, x, hidden=None):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        if len(x.size()) != 3:
            raise RuntimeError(f"Invalid dimensions {x.size()} for the input tensor. "
                               "A 3D tensor with dimensions [batch_size, sequence_length, n_features] is expected.")
        if hidden is None:
            hidden = x.new_zeros(self.n_layers, x.size(0), 2 * self.hidden_size, requires_grad=False).chunk(chunks=2, dim=2)
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        out = x.transpose(0, 1)
        new_hidden = torch.empty_like(hidden[0])
        new_cell = torch.empty_like(hidden[1])
        for l in range(self.n_layers):
            outs = []
            h = hidden[0][l]
            c = hidden[1][l]
            for step_batch in out:
                out, (h, c) = self.__getattr__(f"peephole_lstm{l}")(step_batch, (h, c))
                outs.append(out)
            out = torch.stack(outs)
            new_hidden[l] = h
            new_cell[l] = c

        out = out.transpose(0, 1).contiguous()

        out = self.fc(out)
        hidden = (new_hidden, new_cell)
        
        # return the final output and the hidden state
        return out, hidden
    
    def init_hidden(self, batch_size, cuda=True):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if cuda:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [13]:
class LNPeepholeLSTMTorch(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False, dropout=0., dropout_on_output=True, eps=1e-05):
        if not 0 <= dropout <= 1:
            raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")
        super(LNPeepholeLSTMTorch, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_first = bool(batch_first)
        self.dropout = float(dropout)
        self.dropout_on_output = bool(dropout_on_output)
        self.eps = eps

        self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
        self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
        self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size)))
        self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))

        self.register_parameter('gamma_f', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('gamma_i', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('gamma_g', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('gamma_o', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('gamma_cell', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('beta_cell', nn.Parameter(torch.empty(hidden_size)))

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
        self.weight_ih.data.uniform_(-stdv, +stdv)
        self.weight_hh.data.uniform_(-stdv, +stdv)
        self.weight_ch.data.uniform_(-stdv, +stdv)

        self.bias.data.zero_()
        self.bias.data[:self.hidden_size].fill_(1.)

        self.gamma_f.data.uniform_()
        self.gamma_i.data.uniform_()
        self.gamma_g.data.uniform_()
        self.gamma_o.data.uniform_()
        self.gamma_cell.data.uniform_()
        self.beta_cell.data.zero_()

    def forward(self, input, states):
        assert input.dim() == 3, "expected a 3 dimensional tensor as `input`, but te given tensor has {} dimension(s)".format(input.dim())
        assert len(states) == 2, "expected a (hidden, cell) pair as `states`, but the length of the given states is {}".format(len(states))
        if self.batch_first:
            input = input.transpose(0, 1).contiguous()
        assert states[0].size() == (input.size(1), self.hidden_size), "expected a hidden state tensor with dimensionality {}, but the given tensor has dimensionality []".format(states[0].size(), (input.size(1), self.hidden_size))
        assert states[1].size() == (input.size(1), self.hidden_size), "expected a cell state tensor with dimensionality {}, but the given tensor has dimensionality []".format(states[1].size(), (input.size(1), self.hidden_size))

        hidden, cell = states

        hidden_size = self.hidden_size
        hidden_size_2 = 2 * hidden_size
        hidden_size_3 = hidden_size_2 + hidden_size

        norm_shape = torch.Size((hidden_size,))

        outputs = input.new_empty((input.size(0), input.size(1), hidden_size))

        ih = input.matmul(self.weight_ih.t())

        weight_hc_h = torch.cat((self.weight_hh.t(),
                                 torch.cat((self.weight_ch[:hidden_size].diag(),
                                            self.weight_ch[hidden_size:hidden_size_2].diag(),
                                            self.weight_ch.new_zeros(hidden_size_2, hidden_size))).t()))
        weight_co = self.weight_ch[hidden_size_2:]

        gamma_fig = torch.stack((self.gamma_f, self.gamma_i, self.gamma_g))

        bias_fig = torch.stack(self.bias[:hidden_size_3].chunk(3, dim=0))
        bias_o = self.bias[hidden_size_3:]

        for i in range(input.size(0)):
            gates = torch.addmm(ih[i], torch.cat((hidden, cell), dim=1), weight_hc_h).view(-1, 4, hidden_size)
            gates_fig = gates[:, :3]


            gates_fig = F.layer_norm(gates_fig, norm_shape, eps=self.eps)
            gates_fig = torch.addcmul(bias_fig, gates_fig, gamma_fig)
            forget_input_gates = gates_fig[:, :2].sigmoid()
            candidate_cell = F.dropout(gates_fig[:, 2].tanh(), p=self.dropout, training=self.training)


            cell = F.layer_norm(torch.addcmul(forget_input_gates[:, 0] * cell,
                                              forget_input_gates[:, 1], candidate_cell),
                                norm_shape, self.gamma_cell, self.beta_cell, self.eps)

            output_gate = torch.addcmul(gates[:, 3], cell, weight_co)

            output_gate = F.layer_norm(output_gate, norm_shape, self.gamma_o, bias_o, self.eps).sigmoid()

            hidden = output_gate * cell.tanh()

            outputs[i] = hidden

        if self.dropout_on_output:
            outputs = F.dropout(outputs, p=self.dropout, training=self.training)
        
        if self.batch_first:
            outputs = outputs.transpose(0, 1).contiguous()

        return outputs, (hidden, cell)

    def __repr__(self):
        return f"LNPeepholeLSTMTorch(input_size={self.input_size}, hidden_size={self.hidden_size}, batch_first={self.batch_first}, dropout={self.dropout}, dropout_on_output={self.dropout_on_output}, eps={self.eps})"

from ln_lstm_layer.ln_lstm_layer import LNPeepholeLSTM

C:\Users\0107w\Anaconda3\envs\pytorch1.0\lib\site-packages\torch\utils\cpp_extension.py:184: UserWarning: Error checking compiler version for c++: Command 'c++' returned non-zero exit status 1.
  warnings.warn('Error checking compiler version for {}: {}'.format(compiler, error))


In [14]:
module = lambda **kwds: LNPeepholeLSTM(**kwds, batch_first=True)
# module = lambda **kwds: LNPeepholeLSTMTorch(**kwds, batch_first=True)
# module = lambda **kwds: nn.LSTM(**kwds, batch_first=True)
# module = lambda **kwds: nn.GRU(**kwds, batch_first=True)
# module = peephole_lstm_classes.PeepholeNormLSTMTorch
# module = peephole_lstm_classes.PeepholeNormLSTMTorch
class MyCharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the layers of the model
        self.fc0 = nn.Linear(in_features=len(tokens), out_features=len(tokens))
        self.rnn0 = module(input_size=len(tokens), hidden_size=n_hidden, dropout=drop_prob)
        self.rnn_type = type(self.rnn0)
        for n in range(1, n_layers):
            self.add_module(f"rnn{n}", module(input_size=n_hidden, hidden_size=n_hidden, dropout=drop_prob))
        self.fc1 = nn.Linear(in_features=n_hidden, out_features=len(tokens))
        self.hidden0 = nn.Parameter(torch.randn(n_layers, n_hidden) * 0.1)
        self.cell0 = nn.Parameter(torch.randn(n_layers, n_hidden) * 0.1)
    
    def forward(self, x, hidden=None):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        if len(x.size()) != 3:
            raise RuntimeError(f"Invalid dimensions {x.size()} for the input tensor. "
                               "A 3D tensor with dimensions [batch_size, sequence_length, n_features] is expected.")
        if hidden is None:
            hidden = self.init_hidden(x.size(0))
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        if self.rnn_type in (nn.RNN, nn.GRU):
            new_hidden = torch.empty_like(hidden)
        else:
            new_hidden = torch.empty_like(hidden[0])
            new_cell = torch.empty_like(hidden[1])
        
        x = F.relu(self.fc0(x))
        outputs = []
        for l in range(self.n_layers):
            if self.rnn_type == nn.LSTM:
                x, (new_hidden[l], new_cell[l]) = getattr(self, "rnn{}".format(l))(x, (hidden[0][l].unsqueeze(0), hidden[1][l].unsqueeze(0)))
            elif self.rnn_type in (nn.RNN, nn.GRU):
                x, new_hidden[l] = getattr(self, "rnn{}".format(l))(x, hidden[l].unsqueeze(0))
            else:
                x, (new_hidden[l], new_cell[l]) = getattr(self, "rnn{}".format(l))(x, (hidden[0][l], hidden[1][l]))
            outputs.append(x)
    
        x = self.fc1(sum(outputs))

        # return the final output and the hidden state
        if self.rnn_type in (nn.RNN, nn.GRU):
            return x, new_hidden
        return x, (new_hidden, new_cell)
    
    def init_hidden(self, batch_size, cuda=None):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
#         hidden = (self.hidden0.unsqueeze(1).expand(self.n_layers, batch_size, self.n_hidden).contiguous(),
#                   self.cell0.unsqueeze(1).expand(self.n_layers, batch_size, self.n_hidden).contiguous())
        weight = next(self.parameters()).data
        
        if cuda is None:
            if self.rnn_type in (nn.RNN, nn.GRU):
                hidden = weight.new_zeros((self.n_layers, batch_size, self.n_hidden))
            else:
                hidden = (weight.new_zeros((self.n_layers, batch_size, self.n_hidden)),
                          weight.new_zeros((self.n_layers, batch_size, self.n_hidden)))
        elif cuda:
            if self.rnn_type in (nn.RNN, nn.GRU):
                hidden = weight.new_zeros((self.n_layers, batch_size, self.n_hidden), device='cude')
            else:
                hidden = (weight.new_zeros((self.n_layers, batch_size, self.n_hidden), device='cuda'),
                          weight.new_zeros((self.n_layers, batch_size, self.n_hidden), device='cuda'))
        else:
            if self.rnn_type in (nn.RNN, nn.GRU):
                hidden = weight.new_zeros((self.n_layers, batch_size, self.n_hidden), device='cpu')
            else:
                hidden = (weight.new_zeros((self.n_layers, batch_size, self.n_hidden), device='cpu'),
                          weight.new_zeros((self.n_layers, batch_size, self.n_hidden), device='cpu'))
        
        return hidden

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [15]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, weight_decay=0., clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if (train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if train_on_gpu:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            if isinstance(h, (tuple, list)): #LSTM
                h = tuple([each.data for each in h])
            else: #RNN/GRU
                h = h.data

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output.view(batch_size*seq_length, output.shape[-1]), targets.contiguous().view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
#                 # Get validation loss
#                 val_h = net.init_hidden(batch_size, train_on_gpu)
#                 val_losses = []
#                 net.eval()
#                 for x, y in get_batches(val_data, batch_size, seq_length):
#                     # One-hot encode our data and make them Torch tensors
#                     x = one_hot_encode(x, n_chars)
#                     x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
#                     # Creating new variables for the hidden state, otherwise
#                     # we'd backprop through the entire training history
#                     val_h = tuple([each.data for each in val_h])
                    
#                     inputs, targets = x, y
#                     if(train_on_gpu):
#                         inputs, targets = inputs.cuda(), targets.cuda()

#                     output, val_h = net(inputs, val_h)
#                     val_loss = criterion(output.view(batch_size*seq_length, output.shape[-1]), targets.contiguous().view(batch_size*seq_length).long())
                
#                     val_losses.append(val_loss.item())
                
#                 net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()))
#                       "Val Loss: {:.4f}".format(np.mean(val_losses)))
    torch.cuda.empty_cache()

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [16]:
## TODO: set you model hyperparameters
# define and print the net
n_hidden=240
n_layers=3

# net = CharRNN(chars, n_hidden, n_layers, drop_prob=0.5, lr=0.0001)
net = MyCharRNN(chars, n_hidden, n_layers, drop_prob=0., lr=0.001)
print(net)

MyCharRNN(
  (fc0): Linear(in_features=83, out_features=83, bias=True)
  (rnn0): LNPeepholeLSTM(input_size=83, hidden_size=240, batch_first=True, dropout=0.0, dropout_on_output=True, eps=1e-05)
  (rnn1): LNPeepholeLSTM(input_size=240, hidden_size=240, batch_first=True, dropout=0.0, dropout_on_output=True, eps=1e-05)
  (rnn2): LNPeepholeLSTM(input_size=240, hidden_size=240, batch_first=True, dropout=0.0, dropout_on_output=True, eps=1e-05)
  (fc1): Linear(in_features=240, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [17]:
batch_size = 96
seq_length = 500
n_epochs = 100 # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, weight_decay=0, clip=5, print_every=1)

Epoch: 1/100... Step: 1... Loss: 4.4503...
Epoch: 1/100... Step: 2... Loss: 4.4049...
Epoch: 1/100... Step: 3... Loss: 4.3492...
Epoch: 1/100... Step: 4... Loss: 4.2350...
Epoch: 1/100... Step: 5... Loss: 4.1805...
Epoch: 1/100... Step: 6... Loss: 4.0208...
Epoch: 1/100... Step: 7... Loss: 3.8852...
Epoch: 1/100... Step: 8... Loss: 3.7353...
Epoch: 1/100... Step: 9... Loss: 3.6035...
Epoch: 1/100... Step: 10... Loss: 3.5070...
Epoch: 1/100... Step: 11... Loss: 3.4146...
Epoch: 1/100... Step: 12... Loss: 3.3420...
Epoch: 1/100... Step: 13... Loss: 3.3003...
Epoch: 1/100... Step: 14... Loss: 3.2572...
Epoch: 1/100... Step: 15... Loss: 3.2073...
Epoch: 1/100... Step: 16... Loss: 3.1973...
Epoch: 1/100... Step: 17... Loss: 3.1848...
Epoch: 1/100... Step: 18... Loss: 3.1738...
Epoch: 1/100... Step: 19... Loss: 3.1525...
Epoch: 1/100... Step: 20... Loss: 3.1416...
Epoch: 1/100... Step: 21... Loss: 3.1306...
Epoch: 1/100... Step: 22... Loss: 3.1393...
Epoch: 1/100... Step: 23... Loss: 3.1327.

Epoch: 6/100... Step: 186... Loss: 2.2278...
Epoch: 6/100... Step: 187... Loss: 2.2107...
Epoch: 6/100... Step: 188... Loss: 2.2034...
Epoch: 6/100... Step: 189... Loss: 2.1995...
Epoch: 6/100... Step: 190... Loss: 2.1778...
Epoch: 6/100... Step: 191... Loss: 2.1974...
Epoch: 6/100... Step: 192... Loss: 2.1667...
Epoch: 6/100... Step: 193... Loss: 2.2005...
Epoch: 6/100... Step: 194... Loss: 2.1674...
Epoch: 6/100... Step: 195... Loss: 2.1597...
Epoch: 6/100... Step: 196... Loss: 2.1366...
Epoch: 6/100... Step: 197... Loss: 2.1259...
Epoch: 6/100... Step: 198... Loss: 2.1263...
Epoch: 6/100... Step: 199... Loss: 2.1364...
Epoch: 6/100... Step: 200... Loss: 2.1227...
Epoch: 6/100... Step: 201... Loss: 2.1339...
Epoch: 6/100... Step: 202... Loss: 2.1444...
Epoch: 6/100... Step: 203... Loss: 2.1297...
Epoch: 6/100... Step: 204... Loss: 2.1196...
Epoch: 6/100... Step: 205... Loss: 2.1319...
Epoch: 6/100... Step: 206... Loss: 2.0991...
Epoch: 6/100... Step: 207... Loss: 2.1151...
Epoch: 6/1

Epoch: 10/100... Step: 368... Loss: 1.4781...
Epoch: 10/100... Step: 369... Loss: 1.4690...
Epoch: 10/100... Step: 370... Loss: 1.4658...
Epoch: 11/100... Step: 371... Loss: 1.4889...
Epoch: 11/100... Step: 372... Loss: 1.4767...
Epoch: 11/100... Step: 373... Loss: 1.4654...
Epoch: 11/100... Step: 374... Loss: 1.4691...
Epoch: 11/100... Step: 375... Loss: 1.4443...
Epoch: 11/100... Step: 376... Loss: 1.4611...
Epoch: 11/100... Step: 377... Loss: 1.4477...
Epoch: 11/100... Step: 378... Loss: 1.4661...
Epoch: 11/100... Step: 379... Loss: 1.4637...
Epoch: 11/100... Step: 380... Loss: 1.4550...
Epoch: 11/100... Step: 381... Loss: 1.4293...
Epoch: 11/100... Step: 382... Loss: 1.4125...
Epoch: 11/100... Step: 383... Loss: 1.4133...
Epoch: 11/100... Step: 384... Loss: 1.4257...
Epoch: 11/100... Step: 385... Loss: 1.4255...
Epoch: 11/100... Step: 386... Loss: 1.4116...
Epoch: 11/100... Step: 387... Loss: 1.4068...
Epoch: 11/100... Step: 388... Loss: 1.4232...
Epoch: 11/100... Step: 389... Loss

Epoch: 15/100... Step: 547... Loss: 1.2400...
Epoch: 15/100... Step: 548... Loss: 1.2301...
Epoch: 15/100... Step: 549... Loss: 1.2370...
Epoch: 15/100... Step: 550... Loss: 1.2174...
Epoch: 15/100... Step: 551... Loss: 1.2362...
Epoch: 15/100... Step: 552... Loss: 1.1982...
Epoch: 15/100... Step: 553... Loss: 1.2397...
Epoch: 15/100... Step: 554... Loss: 1.2208...
Epoch: 15/100... Step: 555... Loss: 1.2170...
Epoch: 16/100... Step: 556... Loss: 1.2518...
Epoch: 16/100... Step: 557... Loss: 1.2253...
Epoch: 16/100... Step: 558... Loss: 1.2222...
Epoch: 16/100... Step: 559... Loss: 1.2331...
Epoch: 16/100... Step: 560... Loss: 1.2145...
Epoch: 16/100... Step: 561... Loss: 1.2259...
Epoch: 16/100... Step: 562... Loss: 1.2164...
Epoch: 16/100... Step: 563... Loss: 1.2339...
Epoch: 16/100... Step: 564... Loss: 1.2358...
Epoch: 16/100... Step: 565... Loss: 1.2281...
Epoch: 16/100... Step: 566... Loss: 1.2075...
Epoch: 16/100... Step: 567... Loss: 1.1919...
Epoch: 16/100... Step: 568... Loss

Epoch: 20/100... Step: 726... Loss: 1.1224...
Epoch: 20/100... Step: 727... Loss: 1.1387...
Epoch: 20/100... Step: 728... Loss: 1.1914...
Epoch: 20/100... Step: 729... Loss: 1.1659...
Epoch: 20/100... Step: 730... Loss: 1.1566...
Epoch: 20/100... Step: 731... Loss: 1.1426...
Epoch: 20/100... Step: 732... Loss: 1.1686...
Epoch: 20/100... Step: 733... Loss: 1.1609...
Epoch: 20/100... Step: 734... Loss: 1.1611...
Epoch: 20/100... Step: 735... Loss: 1.1486...
Epoch: 20/100... Step: 736... Loss: 1.1608...
Epoch: 20/100... Step: 737... Loss: 1.1270...
Epoch: 20/100... Step: 738... Loss: 1.1631...
Epoch: 20/100... Step: 739... Loss: 1.1333...
Epoch: 20/100... Step: 740... Loss: 1.1293...
Epoch: 21/100... Step: 741... Loss: 1.1603...
Epoch: 21/100... Step: 742... Loss: 1.1399...
Epoch: 21/100... Step: 743... Loss: 1.1365...
Epoch: 21/100... Step: 744... Loss: 1.1447...
Epoch: 21/100... Step: 745... Loss: 1.1334...
Epoch: 21/100... Step: 746... Loss: 1.1390...
Epoch: 21/100... Step: 747... Loss

...

Epoch: 77/100... Step: 2832... Loss: 0.7961...
Epoch: 77/100... Step: 2833... Loss: 0.7891...
Epoch: 77/100... Step: 2834... Loss: 0.7956...
Epoch: 77/100... Step: 2835... Loss: 0.7871...
Epoch: 77/100... Step: 2836... Loss: 0.7862...
Epoch: 77/100... Step: 2837... Loss: 0.7900...
Epoch: 77/100... Step: 2838... Loss: 0.7848...
Epoch: 77/100... Step: 2839... Loss: 0.7935...
Epoch: 77/100... Step: 2840... Loss: 0.7831...
Epoch: 77/100... Step: 2841... Loss: 0.7980...
Epoch: 77/100... Step: 2842... Loss: 0.7925...
Epoch: 77/100... Step: 2843... Loss: 0.7929...
Epoch: 77/100... Step: 2844... Loss: 0.7944...
Epoch: 77/100... Step: 2845... Loss: 0.7974...
Epoch: 77/100... Step: 2846... Loss: 0.7818...
Epoch: 77/100... Step: 2847... Loss: 0.8012...
Epoch: 77/100... Step: 2848... Loss: 0.7815...
Epoch: 77/100... Step: 2849... Loss: 0.7826...
Epoch: 78/100... Step: 2850... Loss: 0.7984...
Epoch: 78/100... Step: 2851... Loss: 0.7888...
Epoch: 78/100... Step: 2852... Loss: 0.7857...
Epoch: 78/100

Epoch: 82/100... Step: 3007... Loss: 0.7992...
Epoch: 82/100... Step: 3008... Loss: 0.7767...
Epoch: 82/100... Step: 3009... Loss: 0.7633...
Epoch: 82/100... Step: 3010... Loss: 0.7670...
Epoch: 82/100... Step: 3011... Loss: 0.7821...
Epoch: 82/100... Step: 3012... Loss: 0.7794...
Epoch: 82/100... Step: 3013... Loss: 0.7753...
Epoch: 82/100... Step: 3014... Loss: 0.7780...
Epoch: 82/100... Step: 3015... Loss: 0.7831...
Epoch: 82/100... Step: 3016... Loss: 0.7685...
Epoch: 82/100... Step: 3017... Loss: 0.7771...
Epoch: 82/100... Step: 3018... Loss: 0.7732...
Epoch: 82/100... Step: 3019... Loss: 0.7807...
Epoch: 82/100... Step: 3020... Loss: 0.7723...
Epoch: 82/100... Step: 3021... Loss: 0.7741...
Epoch: 82/100... Step: 3022... Loss: 0.7879...
Epoch: 82/100... Step: 3023... Loss: 0.7760...
Epoch: 82/100... Step: 3024... Loss: 0.7830...
Epoch: 82/100... Step: 3025... Loss: 0.7702...
Epoch: 82/100... Step: 3026... Loss: 0.7815...
Epoch: 82/100... Step: 3027... Loss: 0.7809...
Epoch: 82/100

Epoch: 86/100... Step: 3182... Loss: 0.7703...
Epoch: 87/100... Step: 3183... Loss: 0.7731...
Epoch: 87/100... Step: 3184... Loss: 0.7633...
Epoch: 87/100... Step: 3185... Loss: 0.7571...
Epoch: 87/100... Step: 3186... Loss: 0.7657...
Epoch: 87/100... Step: 3187... Loss: 0.7705...
Epoch: 87/100... Step: 3188... Loss: 0.7703...
Epoch: 87/100... Step: 3189... Loss: 0.7637...
Epoch: 87/100... Step: 3190... Loss: 0.7769...
Epoch: 87/100... Step: 3191... Loss: 0.7819...
Epoch: 87/100... Step: 3192... Loss: 0.7813...
Epoch: 87/100... Step: 3193... Loss: 0.7627...
Epoch: 87/100... Step: 3194... Loss: 0.7544...
Epoch: 87/100... Step: 3195... Loss: 0.7601...
Epoch: 87/100... Step: 3196... Loss: 0.7698...
Epoch: 87/100... Step: 3197... Loss: 0.7651...
Epoch: 87/100... Step: 3198... Loss: 0.7580...
Epoch: 87/100... Step: 3199... Loss: 0.7606...
Epoch: 87/100... Step: 3200... Loss: 0.7649...
Epoch: 87/100... Step: 3201... Loss: 0.7548...
Epoch: 87/100... Step: 3202... Loss: 0.7636...
Epoch: 87/100

Epoch: 91/100... Step: 3357... Loss: 0.7403...
Epoch: 91/100... Step: 3358... Loss: 0.7362...
Epoch: 91/100... Step: 3359... Loss: 0.7586...
Epoch: 91/100... Step: 3360... Loss: 0.7471...
Epoch: 91/100... Step: 3361... Loss: 0.7492...
Epoch: 91/100... Step: 3362... Loss: 0.7502...
Epoch: 91/100... Step: 3363... Loss: 0.7493...
Epoch: 91/100... Step: 3364... Loss: 0.7372...
Epoch: 91/100... Step: 3365... Loss: 0.7603...
Epoch: 91/100... Step: 3366... Loss: 0.7481...
Epoch: 91/100... Step: 3367... Loss: 0.7404...
Epoch: 92/100... Step: 3368... Loss: 0.7502...
Epoch: 92/100... Step: 3369... Loss: 0.7453...
Epoch: 92/100... Step: 3370... Loss: 0.7368...
Epoch: 92/100... Step: 3371... Loss: 0.7474...
Epoch: 92/100... Step: 3372... Loss: 0.7482...
Epoch: 92/100... Step: 3373... Loss: 0.7451...
Epoch: 92/100... Step: 3374... Loss: 0.7479...
Epoch: 92/100... Step: 3375... Loss: 0.7609...
Epoch: 92/100... Step: 3376... Loss: 0.7555...
Epoch: 92/100... Step: 3377... Loss: 0.7559...
Epoch: 92/100

Epoch: 96/100... Step: 3532... Loss: 0.7293...
Epoch: 96/100... Step: 3533... Loss: 0.7309...
Epoch: 96/100... Step: 3534... Loss: 0.7192...
Epoch: 96/100... Step: 3535... Loss: 0.7293...
Epoch: 96/100... Step: 3536... Loss: 0.7294...
Epoch: 96/100... Step: 3537... Loss: 0.7255...
Epoch: 96/100... Step: 3538... Loss: 0.7157...
Epoch: 96/100... Step: 3539... Loss: 0.7177...
Epoch: 96/100... Step: 3540... Loss: 0.7252...
Epoch: 96/100... Step: 3541... Loss: 0.7118...
Epoch: 96/100... Step: 3542... Loss: 0.7141...
Epoch: 96/100... Step: 3543... Loss: 0.7031...
Epoch: 96/100... Step: 3544... Loss: 0.7198...
Epoch: 96/100... Step: 3545... Loss: 0.7075...
Epoch: 96/100... Step: 3546... Loss: 0.7100...
Epoch: 96/100... Step: 3547... Loss: 0.7108...
Epoch: 96/100... Step: 3548... Loss: 0.7164...
Epoch: 96/100... Step: 3549... Loss: 0.7029...
Epoch: 96/100... Step: 3550... Loss: 0.7213...
Epoch: 96/100... Step: 3551... Loss: 0.7039...
Epoch: 96/100... Step: 3552... Loss: 0.7039...
Epoch: 97/100

In [41]:
sleep(3)
torch.cuda.empty_cache()

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [18]:
# change the name, for saving multiple files
model_name = 'LNPeepholeLSTM100.pth'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [19]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        if isinstance(h, (list, tuple)):
            h = tuple([each.data for each in h])
        else:
            h = h.data
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out.view(-1, out.shape[-1]), dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [45]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    with torch.no_grad():
        # First off, run through the prime characters
        chars = [ch for ch in prime]
        char_ints = [net.char2int[ch] for ch in prime[:-1]]
        char_arr = one_hot_encode(np.array([char_ints]), len(net.chars))
        inputs = torch.from_numpy(char_arr)
        if train_on_gpu:
            inputs = inputs.cuda()
        h = net.init_hidden(1)
        _, h = net(inputs, h)
    
        # Now pass in the previous character and get a new one
        for ii in tqdm(iterable=range(size), desc="Generating Character by Character...", unit="chr"):
            char, h = predict(net, chars[-1], h, top_k=top_k)
            chars.append(char)

    return ''.join(chars)

In [46]:
train_on_gpu = True
generated_text = sample(net, 100000, prime=generated_text, top_k=3)
print(generated_text)




Generating Character by Character...:   0%|                                                | 0/100000 [00:00<?, ?chr/s]


Generating Character by Character...:   0%|                                      | 12/100000 [00:00<13:53, 119.97chr/s]


Generating Character by Character...:   0%|                                      | 26/100000 [00:00<13:19, 125.02chr/s]


Generating Character by Character...:   0%|                                      | 40/100000 [00:00<12:56, 128.81chr/s]


Generating Character by Character...:   0%|                                      | 54/100000 [00:00<12:39, 131.60chr/s]


Generating Character by Character...:   0%|                                      | 67/100000 [00:00<12:51, 129.55chr/s]


Generating Character by Character...:   0%|                                      | 81/100000 [00:00<12:42, 131.03chr/s]


Generating Character by Character...:   0%|                                      | 95/100000 [00:00<12:40, 131.34chr/s]


Generating Character 

Generating Character by Character...:   1%|▎                                    | 888/100000 [00:07<13:47, 119.82chr/s]


Generating Character by Character...:   1%|▎                                    | 902/100000 [00:07<13:28, 122.60chr/s]


Generating Character by Character...:   1%|▎                                    | 916/100000 [00:07<13:06, 125.99chr/s]


Generating Character by Character...:   1%|▎                                    | 930/100000 [00:07<12:51, 128.46chr/s]


Generating Character by Character...:   1%|▎                                    | 944/100000 [00:07<12:36, 130.97chr/s]


Generating Character by Character...:   1%|▎                                    | 958/100000 [00:07<12:27, 132.42chr/s]


Generating Character by Character...:   1%|▎                                    | 972/100000 [00:07<12:19, 133.83chr/s]


Generating Character by Character...:   1%|▎                                    | 986/100000 [00:08<12:37, 130.69chr/s]


Generating Character by 

Generating Character by Character...:   2%|▋                                   | 1832/100000 [00:14<11:16, 145.05chr/s]


Generating Character by Character...:   2%|▋                                   | 1847/100000 [00:14<11:15, 145.23chr/s]


Generating Character by Character...:   2%|▋                                   | 1862/100000 [00:14<11:15, 145.35chr/s]


Generating Character by Character...:   2%|▋                                   | 1877/100000 [00:14<11:16, 144.99chr/s]


Generating Character by Character...:   2%|▋                                   | 1892/100000 [00:15<11:25, 143.12chr/s]


Generating Character by Character...:   2%|▋                                   | 1907/100000 [00:15<12:36, 129.67chr/s]


Generating Character by Character...:   2%|▋                                   | 1922/100000 [00:15<12:17, 133.03chr/s]


Generating Character by Character...:   2%|▋                                   | 1937/100000 [00:15<12:05, 135.10chr/s]


Generating Character by 

Generating Character by Character...:   3%|█                                   | 2827/100000 [00:21<13:13, 122.53chr/s]


Generating Character by Character...:   3%|█                                   | 2842/100000 [00:21<12:35, 128.65chr/s]


Generating Character by Character...:   3%|█                                   | 2857/100000 [00:22<12:06, 133.66chr/s]


Generating Character by Character...:   3%|█                                   | 2872/100000 [00:22<11:50, 136.67chr/s]


Generating Character by Character...:   3%|█                                   | 2886/100000 [00:22<11:47, 137.24chr/s]


Generating Character by Character...:   3%|█                                   | 2901/100000 [00:22<11:41, 138.50chr/s]


Generating Character by Character...:   3%|█                                   | 2916/100000 [00:22<11:34, 139.78chr/s]


Generating Character by Character...:   3%|█                                   | 2931/100000 [00:22<11:29, 140.69chr/s]


Generating Character by 

Generating Character by Character...:   4%|█▎                                  | 3787/100000 [00:29<12:41, 126.28chr/s]


Generating Character by Character...:   4%|█▎                                  | 3800/100000 [00:29<12:39, 126.63chr/s]


Generating Character by Character...:   4%|█▎                                  | 3813/100000 [00:29<13:51, 115.70chr/s]


Generating Character by Character...:   4%|█▍                                  | 3827/100000 [00:29<13:14, 121.10chr/s]


Generating Character by Character...:   4%|█▍                                  | 3840/100000 [00:29<12:59, 123.29chr/s]


Generating Character by Character...:   4%|█▍                                  | 3854/100000 [00:29<12:46, 125.46chr/s]


Generating Character by Character...:   4%|█▍                                  | 3868/100000 [00:29<12:24, 129.14chr/s]


Generating Character by Character...:   4%|█▍                                  | 3883/100000 [00:29<12:00, 133.32chr/s]


Generating Character by 

...

Generating Character by Character...:  97%|██████████████████████████████████ | 97222/100000 [11:50<00:21, 130.71chr/s]


Generating Character by Character...:  97%|██████████████████████████████████ | 97236/100000 [11:50<00:20, 132.24chr/s]


Generating Character by Character...:  97%|██████████████████████████████████ | 97251/100000 [11:50<00:20, 135.26chr/s]


Generating Character by Character...:  97%|██████████████████████████████████ | 97266/100000 [11:50<00:19, 138.60chr/s]


Generating Character by Character...:  97%|██████████████████████████████████ | 97281/100000 [11:50<00:19, 140.64chr/s]


Generating Character by Character...:  97%|██████████████████████████████████ | 97296/100000 [11:51<00:19, 142.10chr/s]


Generating Character by Character...:  97%|██████████████████████████████████ | 97311/100000 [11:51<00:18, 143.14chr/s]


Generating Character by Character...:  97%|██████████████████████████████████ | 97326/100000 [11:51<00:18, 143.88chr/s]


Generating Character by 

Generating Character by Character...:  98%|██████████████████████████████████▎| 98206/100000 [11:57<00:12, 144.40chr/s]


Generating Character by Character...:  98%|██████████████████████████████████▍| 98221/100000 [11:57<00:12, 144.76chr/s]


Generating Character by Character...:  98%|██████████████████████████████████▍| 98236/100000 [11:57<00:12, 145.02chr/s]


Generating Character by Character...:  98%|██████████████████████████████████▍| 98251/100000 [11:58<00:12, 145.63chr/s]


Generating Character by Character...:  98%|██████████████████████████████████▍| 98266/100000 [11:58<00:12, 140.71chr/s]


Generating Character by Character...:  98%|██████████████████████████████████▍| 98281/100000 [11:58<00:12, 141.35chr/s]


Generating Character by Character...:  98%|██████████████████████████████████▍| 98296/100000 [11:58<00:11, 142.61chr/s]


Generating Character by Character...:  98%|██████████████████████████████████▍| 98311/100000 [11:58<00:11, 142.68chr/s]


Generating Character by 

Generating Character by Character...:  99%|██████████████████████████████████▋| 99206/100000 [12:04<00:05, 145.72chr/s]


Generating Character by Character...:  99%|██████████████████████████████████▋| 99221/100000 [12:04<00:05, 146.12chr/s]


Generating Character by Character...:  99%|██████████████████████████████████▋| 99236/100000 [12:04<00:05, 146.83chr/s]


Generating Character by Character...:  99%|██████████████████████████████████▋| 99251/100000 [12:05<00:05, 141.10chr/s]


Generating Character by Character...:  99%|██████████████████████████████████▋| 99266/100000 [12:05<00:05, 142.43chr/s]


Generating Character by Character...:  99%|██████████████████████████████████▋| 99281/100000 [12:05<00:05, 143.79chr/s]


Generating Character by Character...:  99%|██████████████████████████████████▊| 99296/100000 [12:05<00:04, 144.33chr/s]


Generating Character by Character...:  99%|██████████████████████████████████▊| 99311/100000 [12:05<00:04, 144.72chr/s]


Generating Character by 

"Which didn't I tell us the memory?" he asked Levin.  As soon as the position was, as the money in the same horror he saw was the expression of his father had been talked with her, and, with the solitary gardens lighted been.  Levin went to head with a books, cost stretched up, and stood out into the right he caught the chain. "Ah, I'm not well and wondering all the putting on its country for them. I promise everything. It means to be. It will be taken the pavilion with Madame Karenina, and he won't be, and do that that can't be and him, but I can't help it. I never lost him, of when she had to do one. But you know, but to dance, but you must have the same nose. I had thought other people, but there is nothing existing to morm, my dear they're unconsciously delightful. And that if me I assure, believe it, that he's depressed in people who was so glad that deeiche young wine--seek you think it was.... What do I want?" said Stepan Arkadyevitch, with a good-humored smile. "And so, what on

while which was in the prival were the measures of the peas


## Loading a checkpoint

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))